In [1]:
import numpy as np
import time
import math
import random
from random import randint

# Simulated Annealing

Solve the Sudoku problem with Simulated Annealing. You can design your own algorithm or simply refer to [Metaheuristics_can_solve_Sudoku_puzzles](https://www.researchgate.net/publication/220403361_Metaheuristics_can_solve_Sudoku_puzzles). 

The code provided below starts with making a problem instance and ends by visualizing the running process of SA.

In [2]:
# making a problem instance
def make_grid_python(n):
    grid = np.empty((n**2, n**2), int)
    # print(grid)
    x = 0
    for i in range(n):
        for j in range(n):
            for k in range(n**2):
                grid[n*i+j, k] = x%(n**2) + 1
                x += 1
            x += n
        x += 1
    return grid

def make_grid_numpy(n):
    return np.fromfunction(lambda i, j: (i*n+i//n+j)%(n**2)+1, (n**2, n**2), dtype=int)

# a comparison between native python and numpy
# vary n to see their performances
n = 10
%timeit make_grid_python(n)
%timeit make_grid_numpy(n)

# test
grid = make_grid_python(3)
grid

3.53 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
61.1 µs ± 591 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


array([[1, 2, 3, 4, 5, 6, 7, 8, 9],
       [4, 5, 6, 7, 8, 9, 1, 2, 3],
       [7, 8, 9, 1, 2, 3, 4, 5, 6],
       [2, 3, 4, 5, 6, 7, 8, 9, 1],
       [5, 6, 7, 8, 9, 1, 2, 3, 4],
       [8, 9, 1, 2, 3, 4, 5, 6, 7],
       [3, 4, 5, 6, 7, 8, 9, 1, 2],
       [6, 7, 8, 9, 1, 2, 3, 4, 5],
       [9, 1, 2, 3, 4, 5, 6, 7, 8]])

In [3]:
class Sudoku:
    @classmethod
    def create(cls, n, seed=303):
        rng = np.random.default_rng(seed)
        init_grid = make_grid_numpy(n)
        # print(init_grid)
        # randomly mask out some cells to create a problem instance
        # cells marked by *1* is given and fixed
        mask = rng.integers(0, 2, size=init_grid.shape)
        grid = init_grid*mask
        # print(grid)

        return cls(n, mask, grid, seed)

    def __init__(self, n, mask, grid, seed) -> None:
        self.seed = seed
        self.mask = mask
        self.grid = grid
        self.n = n
        self.all = set(range(1, n**2+1))

    def value(self):
        n = self.n
        value = 0
        for row in range(n * n):
            row_num = set()
            for col in range(n * n):
                if self.grid[row][col] not in row_num:
                    row_num.add(self.grid[row][col])
                else:
                    value -= 1
        for col in range(n * n):
            col_num = set()
            for row in range(n * n):
                if self.grid[row][col] not in col_num:
                    col_num.add(self.grid[row][col])
                else:
                    value -= 1
        return value
    
    def local_search(self):
        grid = self.grid.copy()
        # TODO: apply your neighborhood search operator to get the next stat
        # for i in range(9):
        # for row_district in range(n):
        #     for col_district in range(n):
        row_district = randint(0, n - 1)
        col_district = randint(0, n - 1)
        x1 = randint(0, n - 1) + n * row_district
        y1 = randint(0, n - 1) + n * col_district
        while self.mask[x1][y1] == 1:
            x1 = randint(0, n - 1) + n * row_district
            y1 = randint(0, n - 1) + n * col_district
            # print(x1, y1)
        x2 = randint(0, n - 1) + n * row_district
        y2 = randint(0, n - 1) + n * col_district
        while self.mask[x2][y2] == 1:
            x2 = randint(0, n - 1) + n * row_district
            y2 = randint(0, n - 1) + n * col_district
            # print(x2, y2)
        temp = grid[x1][y1]
        grid[x1][y1] = grid[x2][y2]
        grid[x2][y2] = temp

        next_state = Sudoku(self.n, self.mask, grid, self.seed)

        return next_state


    def init_solution(self):
        rng = np.random.default_rng(self.seed)
        n = self.n
        grid = self.grid.reshape(n, n, n, n).transpose(0, 2, 1, 3)
        for I in np.ndindex(n, n):
            idx = grid[I]==0
            grid[I][idx] = rng.permutation(list(self.all-set(grid[I].flat)))
        return self
        
    def __repr__(self) -> str:
        return self.grid.__repr__()

# test
sudoku = Sudoku.create(3)
sudoku.init_solution()
sudoku, sudoku.value()

(array([[9, 2, 3, 4, 3, 6, 8, 6, 9],
        [4, 5, 1, 7, 8, 9, 1, 2, 3],
        [7, 6, 8, 5, 2, 1, 4, 5, 7],
        [2, 3, 4, 3, 6, 7, 4, 9, 1],
        [5, 6, 7, 9, 1, 5, 5, 3, 8],
        [8, 9, 1, 8, 2, 4, 7, 2, 6],
        [3, 4, 5, 1, 9, 8, 4, 1, 2],
        [6, 7, 8, 7, 2, 4, 3, 6, 5],
        [9, 1, 2, 3, 6, 5, 9, 7, 8]]),
 -33)

In [4]:
def simulated_annealing(initial:Sudoku, schedule, halt, log_interval=200):
    state = initial.init_solution()
    # print(state)
    t = 0           # time step
    T = schedule(t) # temperature
    f = [state.value()] # a recording of values
    while not halt(T):
        T = schedule(t)
        new_state = state.local_search()
        new_value = new_state.value()
        # TODO: implement the replacement here
        max_value = f[-1]
        if new_value > max_value or (math.exp((new_value - max_value)/T) > random.random()):
            state = new_state
            f.append(new_value)
        # update time and temperature
        if t % log_interval == 0:
            print(f"step {t}: T={T}, current_value={state.value()}")
        t += 1
        T = schedule(t)
    print(f"step {t}: T={T}, current_value={state.value()}")
    return state, f

In [9]:
import matplotlib.pyplot as plt

# define your own schedule and halt condition
# run the algorithm on different n with different settings
n = 4
solution, record = simulated_annealing(
    initial=Sudoku.create(n), 
    schedule=lambda t: 0.999**t, 
    halt=lambda T: T<1e-11
)
solution, solution.value()

step 0: T=1.0, current_value=-125
step 200: T=0.8186488294786356, current_value=-89
step 400: T=0.6701859060067401, current_value=-75
step 600: T=0.5486469074854967, current_value=-58
step 800: T=0.4491491486100751, current_value=-53
step 1000: T=0.36769542477096373, current_value=-55
step 1200: T=0.3010134290933991, current_value=-52
step 1400: T=0.2464242913846615, current_value=-48
step 1600: T=0.20173495769715533, current_value=-46
step 1800: T=0.16515008698369826, current_value=-43
step 2000: T=0.13519992539749945, current_value=-37
step 2200: T=0.11068126067226176, current_value=-33
step 2400: T=0.09060908449456684, current_value=-30
step 2600: T=0.07417702096160793, current_value=-27
step 2800: T=0.060724931384432544, current_value=-19
step 3000: T=0.04971239399803616, current_value=-19
step 3200: T=0.04069699315707305, current_value=-19
step 3400: T=0.033316545811337896, current_value=-19
step 3600: T=0.027274551230723105, current_value=-13
step 3800: T=0.02232827943958655, cur

(array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
        [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  1,  2,  3,  4],
        [ 9, 10, 11, 12, 13, 14, 15, 16,  1,  2,  3,  4,  5,  6,  7,  8],
        [13, 14, 15, 16,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 2,  3,  4,  5,  6,  7, 16,  9, 10, 11, 12, 13, 14, 15,  8,  1],
        [ 6,  7,  8, 13, 10, 11, 12,  1, 14, 15, 16,  5,  2,  3,  4,  9],
        [10, 11, 12,  9, 14, 15,  8, 13,  2,  3,  4,  1,  6,  7, 16,  5],
        [14, 15, 16,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  1,  2],
        [ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  1,  2,  3,  4,  5,  6],
        [11, 12, 13, 14, 15, 16,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
        [15, 16,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
        [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  1,  2,  3],
        [ 8,  9, 10, 11, 12, 13, 14, 1

Text(0, 0.5, 'value')

In [10]:
# visualize the curve
plt.plot(record)
plt.xlabel("time step")
plt.ylabel("value")
# record